In [63]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

city_dest_id = {}
city_dest_id['London'] = '-2601889'
# city_dest_id['Liverpool'] = '-2601422'
# city_dest_id['Nottingham'] = '-2604469'
city_dest_id['Paris'] = '-1456928'
city_dest_id['Berlin'] = '-1746443'
city_dest_id['Madrid'] = '-390625'
city_dest_id['Amesterdam'] = '-2140479'
city_dest_id['Zurich'] = '-2554935'
city_dest_id['Rome'] = '-126693'
def get_hotels_objects(adults_number, time_interval, city_name):
    
    def get_hotels_from_page(soup):
        hotels = soup.findAll('div', {'data-testid': 'property-card'})
        hotel_list = []
        for hotel in hotels:
            
            hotel_list.append(hotel)
        return hotel_list

    check_in_date = datetime.now().strftime('%Y-%m-%d')
    check_out_date = (datetime.now() + timedelta(days=time_interval)).strftime('%Y-%m-%d')
    
    # Initialize parameters
    url = 'https://www.booking.com/searchresults.en-gb.html'
    params = {
        'ss': city_name,
        'ssne': city_name,
        'ssne_untouched': city_name,
        'efdco': '1',
        'label': 'gog235jc-1FCAEoggI46AdICVgDaFCIAQGYAQm4ARfIAQzYAQHoAQH4AQKIAgGoAgO4ArDuuaEGwAIB0gIkZmJhYjE4YzAtNDdhMy00MmY1LTk2NWItN2UzOTgyNTk1OWEx2AIF4AIB',
        'aid': '397594',
        'lang': 'en-gb',
        'sb': '1',
        'src_elem': 'sb',
        'src': 'searchresults',
        'dest_id': city_dest_id[city_name],
        'dest_type': 'city',
        'checkin': check_in_date,
        'checkout': check_out_date,
        'group_adults': str(adults_number),
        'no_rooms': '1',
        'group_children': '0',
        'offset': 0  # Start with the first page
    }
    headers = {
        'User-Agent': 'Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.64 Safari/537.36',
        'Accept-Language': 'en-US, en;q=0.5'
    }
    
    # List to store all hotels
    all_hotels = []
    
    # Pagination handling
    while True:
        response = requests.get(url, headers=headers, params=params)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        hotels_on_page = get_hotels_from_page(soup)
        if not hotels_on_page:
            break  # Exit loop if no hotels are found on the current page
    
        all_hotels.extend(hotels_on_page)
        params['offset'] += 1  # Move to the next page
        if params['offset']==20:
            break
    return all_hotels


In [64]:
# all_hotels = []

In [65]:
import concurrent.futures
all_hotels = []
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = []
    for city_id in city_dest_id.keys():
        for i in range(2,4):
            for j in range(2,4):
                futures.append(executor.submit(get_hotels_objects, i, j, city_id))
    for future in concurrent.futures.as_completed(futures):
        new_hotels = future.result()
        all_hotels.extend(new_hotels)

SSLError: HTTPSConnectionPool(host='www.booking.com', port=443): Max retries exceeded with url: /searchresults.en-gb.html?ss=Zurich&ssne=Zurich&ssne_untouched=Zurich&efdco=1&label=gog235jc-1FCAEoggI46AdICVgDaFCIAQGYAQm4ARfIAQzYAQHoAQH4AQKIAgGoAgO4ArDuuaEGwAIB0gIkZmJhYjE4YzAtNDdhMy00MmY1LTk2NWItN2UzOTgyNTk1OWEx2AIF4AIB&aid=397594&lang=en-gb&sb=1&src_elem=sb&src=searchresults&dest_id=-2554935&dest_type=city&checkin=2024-05-27&checkout=2024-05-29&group_adults=3&no_rooms=1&group_children=0&offset=17 (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1002)')))

In [62]:
print(len(all_hotels))

3604


In [52]:
def extract_feature(hotel, tag, key_dict, value_dict):
    element = hotel.find(tag, {key_dict: value_dict})
    try:
        element_text = element.text.strip()
    except:
        element_text = None
    return element_text

In [53]:
def extract_number_of_beds(hotel):
    try:
        element = hotel.find('div', {'class': 'fc367255e6'}).find_all('div', {'class': 'abf093bdfe'})[-1]
        element_text = element.text.strip()
    except:
        element_text = None
    return element_text

In [54]:
def extract_size(hotel):
    try:
        size_info_div = hotel.find('div', {'data-testid': 'property-card-unit-configuration'})
        spans = size_info_div.find_all('span', class_='a21c5c4883')
        element_text = spans[-1].text.strip() if spans else None
        if element_text[-1] ==  '²':
            return element_text
        else:
            return None
    except:
        return None

In [55]:
hotels_data=[]

In [56]:
for hotel in all_hotels:
    # Extract the hotel name
    name = extract_feature(hotel, 'div', 'data-testid', 'title')
    location = extract_feature(hotel, 'span', 'data-testid', 'address')
    price = extract_feature(hotel, 'span', 'data-testid', 'price-and-discounted-price')
    rating_sample = extract_feature(hotel, 'div', 'class', 'a3b8729ab1 d86cee9b25')
    if rating_sample:
        rating = rating_sample.split(' ')[-1]
    else:
        rating = None
    quality = extract_feature(hotel, 'div', 'class', 'a3b8729ab1 e6208ee469 cb2cbb3ccb')
    review = extract_feature(hotel, 'div', 'class', 'abf093bdfe f45d8e4c32 d935416c47')
    beds = extract_number_of_beds(hotel)
    size = extract_size(hotel)
    distance = extract_feature(hotel, 'span', 'data-testid', 'distance').split(' ')[0]
    room_type = extract_feature(hotel, 'h4', 'class', 'abf093bdfe e8f7c070a7')
    nights_and_adults = extract_feature(hotel, 'div', 'class', 'abf093bdfe f45d8e4c32').split(",")
    nights, adults = nights_and_adults[0], nights_and_adults[1]
    free_cancelation = extract_feature(hotel, 'div', 'class', 'abf093bdfe d068504c75')
    
    
    # Append hotes_data with info about hotel
    hotels_data.append({
        'name': name,
        'location': location,
        'price': price,
        'rating': rating,
        'quality' : quality,
        'review' : review,
        'bed' : beds,
        'size' : size,
        'distance from centure' : distance,
        'room_type' : room_type,
        'nights' : nights,
        'adults' : adults,
        'free_canelation' : free_cancelation
    })

In [57]:
print(len(hotels_data))

2162


In [58]:
import pandas as pd
hotels = pd.DataFrame(hotels_data)
df = hotels
duplicate_rows = df[df.duplicated(keep='first')]
print(len(duplicate_rows))

1048
